# https://www.sec.gov/Archives/edgar/full-index/2017/

In [ ]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup as soup
import lxml
import re
import os

proxy_list = pd.read_table('proxy.txt',sep='\t', header=-1)
header_list = pd.read_table('header.txt',sep='\t', header=-1)

def reset_proxy():
    rand_sample = random.sample(range(len(proxy_list)), 1)

    rand_ip = proxy_list.ix[rand_sample][0].values
    rand_port = proxy_list.ix[rand_sample][1].values

    proxy_target = (str(*rand_ip) +':'+  str(*rand_port))
    
    return proxy_target


def reset_header():
    rand_sample = random.sample(range(len(header_list)), 1)

    rand_header = header_list.ix[rand_sample][0].values

    header_target = (str(*rand_header))
    
    return header_target


def get_10k_path(year):
#     qtrs = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
    qtrs = ['QTR4'] # Only consider 4th Querter of year
    columns = ['CompanyName','FormType','CIK','Date','URL']
    base_url = "http://www.sec.gov/Archives/edgar/full-index/%s/"%year
    df_base = pd.DataFrame(columns=columns)
    for qtr in qtrs:
        url = base_url+qtr+'/crawler.idx'
        print(url)
        lines = requests.get(url).text.splitlines()
        records=[]
        for line in lines:
            comnam = line[0:62].strip()
            formt = line[62:74].strip()
            cik = line[74:86].strip()
            date = line[86:98].strip()
            data_url = line[98:].strip()
            records.append(tuple([comnam,formt,cik,date,data_url]))
        df = pd.DataFrame(records, columns=columns)
        df.to_excel('da.xlsx')
        TenK = df[df['FormType'] == '10-K']
        df_base = df_base.append(TenK)        
        del df, TenK, records
    print('Total number of 10-K filings : ',len(df_base))
    return df_base

cannot_crawl = []

def down_10k(df):
    df.reset_index(inplace=True)
    base_url = "http://www.sec.gov/Archives/"
    total_num = len(df)
    for ind, rows in df.iterrows():
        company = rows['CompanyName']
        company = re.sub(r"[\\+/+]", " ", company)
        cik = rows['CIK']
        date = rows['Date']
#         url = base_url + rows['FileName']
        url = rows['URL']
        print(ind+1,'/',total_num, ' : ', url)
        filename = '--'.join([cik,company,date]) + '.txt'
        
        flag = True
        # try crawling three times if error occurs
        while flag:
            time.sleep(1)
            flag_write = True
            headers = {'User-Agent': reset_header()}
            proxy_dict = {'http': reset_proxy()}
            try:
                response = requests.get(url, headers=headers, proxies=proxy_dict)

                if response.status_code == 200:
                    report = soup(response.content.decode() , 'lxml').get_text()
                    report = re.sub(r"\s\s+", " ", report)
                    splitted = re.split(r'[iI]tem\s+\d+\W+', report)
                    for chunk in splitted:
                        if (chunk.lower().strip().startswith('business') and len(chunk) > 100 and flag_write):
                            with open('C:/Users/Rocku/downloads/sec_10k/'+filename, 'w', encoding='utf-8') as f:
                                f.write(chunk)
                            print(filename, ' : downloaded')
                            flag_write=False

                    flag = False
            except:
                with open('C:/Users/Rocku/downloads/sec_10k/'+'raw--'+filename, 'w', encoding='utf-8') as f:
                    f.write(chunk)
                print(filename, ' : raw downloaded')
                cannot_crawl.append([company,cik,date,url])
                flag = False

In [ ]:
directory = 'C:/Users/Rocku/downloads/sec_10k'
if not os.path.exists(directory):
    os.makedirs(directory)

# Get urls of 10-K filings in 2017
idx_data = get_10k_path(2017)

# New

In [ ]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup as soup
import lxml
import re
import os
import concurrent.futures
    
proxy_list = pd.read_table('proxy.txt', sep='\t', header=-1)
header_list = pd.read_table('header.txt', sep='\t', header=-1)


def reset_proxy():
    rand_sample = random.sample(range(len(proxy_list)), 1)

    rand_ip = proxy_list.ix[rand_sample][0].values
    rand_port = proxy_list.ix[rand_sample][1].values

    proxy_target = (str(*rand_ip) + ':' + str(*rand_port))

    return proxy_target


def reset_header():
    rand_sample = random.sample(range(len(header_list)), 1)

    rand_header = header_list.ix[rand_sample][0].values

    header_target = (str(*rand_header))

    return header_target


def get_10k_path(year):
    #     qtrs = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
    qtrs = ['QTR4']  # Only consider 4th Querter of year
    columns = ['CIK', 'CompanyName', 'FormType', 'Date', 'FileName']
    base_url = "http://www.sec.gov/Archives/edgar/full-index/%s/" % year
    df_base = pd.DataFrame(columns=columns)
    for qtr in qtrs:
        url = base_url + qtr + '/master.idx'
        print(url)
        lines = requests.get(url).text.splitlines()
        records = [tuple(line.split('|')) for line in lines[11:]]
        df = pd.DataFrame(records, columns=columns)
        TenK = df[df['FormType'] == '10-K']
        df_base = df_base.append(TenK)
        del df, TenK, records
    print('Total number of 10-K filings : ', len(df_base))
    return df_base


def download_processor(url, filename):
    headers = {'User-Agent': reset_header()}
    proxy_dict = {'http': reset_proxy()}
    response = requests.get(url, headers=headers, proxies=proxy_dict)

    if response.status_code == 200:
        report = soup(response.content.decode(), 'lxml').get_text()
        report = re.sub(r"\s\s+", " ", report)
        splitted = re.split(r'[iI]tem\s+\d+\W+', report)
        flag_write = True
        for chunk in splitted:
            if (chunk.lower().strip().startswith('business') and len(chunk) > 100 and flag_write):
                with open('sec_10k/' + filename, 'w', encoding='utf-8') as f:
                    f.write(chunk)
                print(filename, ' : downloaded')
                flag_write = False
        del splitted, report
    del response
            

def down_10k(df):
    df.reset_index(inplace=True)
    base_url = "http://www.sec.gov/Archives/"
    total_num = len(df)
    
#     executor = concurrent.futures.ThreadPoolExecutor(max_workers=20)
    cannot = []
    for ind, rows in df.iterrows():
        company = rows['CompanyName']
        company = re.sub(r"[\\+/+]", " ", company)
        cik = rows['CIK']
        date = rows['Date']
        url = base_url + rows['FileName']
        print(ind + 1, '/', total_num, ' : ', url)
        filename = '--'.join([cik, company, date]) + '.txt'
        
        flag = True
        times = 1
        # try crawling three times if error occurs
        while flag:
            print(times)
            if times < 3:
                time.sleep(1)
                try:
                    download_processor(url,filename)
#                     executor.submit(download_processor,url, filename)
#                     print("Waiting for threads to finish...")
#                     executor.shutdown(wait=True)
                    flag=False
                except:
                    times += 1
            else:
                flag=False
                cannot.append(tuple([url, filename]))
                

directory = 'sec_10k'
if not os.path.exists(directory):
    os.makedirs(directory)

# Get urls of 10-K filings in 2017
idx_data = get_10k_path(2017)
cannot = down_10k(idx_data)

# Current VER

In [3]:
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup as soup
import lxml
import re
import os
import concurrent.futures
import requests
import os
import errno
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

proxy_list = pd.read_table('proxy.txt', sep='\t', header=-1)
header_list = pd.read_table('header.txt', sep='\t', header=-1)

# DEFAULT_DATA_PATH = 'sec_10k/'
# DEFAULT_DATA_PATH = 'C:/Users/Rocku/Downloads/sec_10k/'
DEFAULT_DATA_PATH = 'D:\seclogdown/'

class SecCrawler():
    ua = UserAgent()
    def __init__(self):
        print("Path of the directory: " + DEFAULT_DATA_PATH)

    def save_in_directory(self, company_code, cik, priorto, doc_list, doc_name_list, filing_type):
        for j in range(len(doc_list)):
            base_url = doc_list[j]
            print(base_url)
            r = requests.get(base_url, {'headers':self.ua.random})
            report = soup(r.content.decode(), 'lxml')
            report.find_all('table')
            report = re.sub(r"\s\s+", " ", report)
            report = re.sub(r"<.*?>", "", report)
            splitted = re.split(r'[iI]tem\s+\d+\W+', report)
            filename = '--'.join([company_code,cik,priorto,doc_name_list[j]])
            flag_write = True
            for chunk in splitted:
                if (chunk.lower().strip().startswith('business') and len(chunk) > 500 and flag_write):
                    with open(DEFAULT_DATA_PATH + filename, "ab") as f:
                        f.write(chunk.encode('ascii', 'ignore'))
                    flag_write = False
                    print(filename, ' downloaded')
            
    def filing_10K(self, company_code, cik, priorto, count):

        # generate the url to crawl
        base_url = "http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="+str(cik)+"&type=10-K&dateb="+str(priorto)+"&owner=exclude&output=xml&count="+str(count)
        r = requests.get(base_url)
        data = r.text
        doc_list, doc_name_list = self.create_document_list(data)

        try:
            self.save_in_directory(company_code, cik, priorto, doc_list, doc_name_list, '10-K')
        except Exception as e:
            print (str(e))


    def create_document_list(self, data):
        soup_obj = BeautifulSoup(data, 'lxml')
        link_list = list()

        # If the link is .htm convert it to .html
        for link in soup_obj.find_all('filinghref'):
            url = link.string
            if link.string.split(".")[len(link.string.split("."))-1] == "htm":
                url += "l"
            link_list.append(url)
        link_list_final = link_list

        doc_list = list()
        doc_name_list = list()
        for k in range(len(link_list_final[:1])):
            required_url = link_list_final[k].replace('-index.html', '')
            txtdoc = required_url + ".txt"
            docname = txtdoc.split("/")[-1]
            doc_list.append(txtdoc)
            doc_name_list.append(docname)
        return doc_list, doc_name_list
    
    
# def reset_proxy():
#     rand_sample = random.sample(range(len(proxy_list)), 1)
#     rand_ip = proxy_list.ix[rand_sample][0].values
#     rand_port = proxy_list.ix[rand_sample][1].values
#     proxy_target = (str(*rand_ip) + ':' + str(*rand_port))
#     return proxy_target


# def reset_header():
#     rand_sample = random.sample(range(len(header_list)), 1)
#     rand_header = header_list.ix[rand_sample][0].values
#     header_target = (str(*rand_header))
#     return header_target


def get_10k_path(year):
    qtrs = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
    columns = ['CIK', 'CompanyName', 'FormType', 'Date', 'FileName']
    base_url = "http://www.sec.gov/Archives/edgar/full-index/%s/" % year
    df_base = pd.DataFrame(columns=columns)
    
    for qtr in qtrs:
        url = base_url + qtr + '/master.idx'
        print(url)
        lines = requests.get(url).text.splitlines()
        records = [tuple(line.split('|')) for line in lines[11:]]
        df = pd.DataFrame(records, columns=columns)
        TenK = df[df['FormType'] == '10-K']
        df_base = df_base.append(TenK)
        del df, TenK, records
    print('Total number of 10-K filings : ', len(df_base))
    return df_base

In [6]:
if not os.path.exists(DEFAULT_DATA_PATH):
    os.makedirs(DEFAULT_DATA_PATH)

# Get urls of 10-K filings in 2017
idx_data = get_10k_path(2010)
idx_data.reset_index(inplace=True)
idx_data.head()

http://www.sec.gov/Archives/edgar/full-index/2010/QTR1/master.idx
http://www.sec.gov/Archives/edgar/full-index/2010/QTR2/master.idx
http://www.sec.gov/Archives/edgar/full-index/2010/QTR3/master.idx
http://www.sec.gov/Archives/edgar/full-index/2010/QTR4/master.idx
Total number of 10-K filings :  9165


,index,CIK,CompanyName,FormType,Date,FileName
0,39,1000180,SANDISK CORP,10-K,2010-02-25,edgar/data/1000180/0001000180-10-000008.txt
1,79,1000209,MEDALLION FINANCIAL CORP,10-K,2010-03-12,edgar/data/1000209/0001193125-10-055581.txt
2,92,1000228,HENRY SCHEIN INC,10-K,2010-02-23,edgar/data/1000228/0001000228-10-000006.txt
3,141,1000229,CORE LABORATORIES N V,10-K,2010-02-19,edgar/data/1000229/0001000229-10-000004.txt
4,155,1000230,OPTICAL CABLE CORP,10-K,2010-01-29,edgar/data/1000230/0001193125-10-017031.txt


In [111]:
base_url = "https://www.sec.gov/Archives/"
url = idx_data.loc[2,'FileName']
url = base_url + url
# print(url)
ua = UserAgent()
# url = 'https://www.sec.gov/Archives/edgar/data/1000209/0001193125-17-082178-index.htm'
print(url)
r = requests.get(url, {'headers':ua.random})
report = soup(r.content.decode(), 'lxml')

https://www.sec.gov/Archives/edgar/data/1000228/0001000228-10-000006.txt


In [113]:
path_to_business = ''
break_flag = False
for tab in report.find_all('table'):
    if break_flag == True:
        break
    else:
        for a_tag in tab.find_all('a'):
            if a_tag is not None:
                if 'business' in list(a_tag.text.lower().strip().split(' ')):
                    print(a_tag['href'], a_tag.text)
                    path_to_business = a_tag['href']
                    break_flag = True
                    break


path_to_business

#item_1 Business


'#item_1'

In [109]:
for tab in report.find_all('table'):
    for a_tag in tab.find_all('a'):
        print(a_tag)

<a href="#pt1_item_1_business">Business</a>
<a href="#pt1_item_1a_risk_factors">Risk Factors</a>
<a href="#pt1_item_1b_sec_comments">Unresolved Staff
    Comments</a>
<a href="#pt1_item_2_property">Properties</a>
<a href="#item_3_legal_proceedings">Legal Proceedings</a>
<a href="#pt1_item_4_submissions">Submission of Matters to a Vote of Security
      Holders</a>
<a href="#pt2_item_5_market_common_equity">Market for Registrant’s Common
      Equity, Related Stockholder Matters and Issuer Purchases of Equity
      Securities</a>
<a href="#pt2_item_6_financial_data">Selected Financial
    Data</a>
<a href="#pt2_item_7_results_of_operations">Management’s Discussion and
      Analysis of Financial Condition and Results of
    Operation</a>
<a href="#pt2_item_7a_quantitative_disclosures">Quantitative and Qualitative
      Disclosures About Market Risk</a>
<a href="#pt2_item_8_financial_statements">Financial Statements and
      Supplementary Data</a>
<a href="#pt2_item_9_disagreements_w_ac

In [58]:
idx_data = idx_data.loc[:2,:]

secCrawler = SecCrawler()
for ind,ex in idx_data.iterrows():
    CompanyName = re.sub(r"\W+", " ", ex['CompanyName']).strip()
    CIK = ex['CIK']
    Date = ex['Date'].replace('-','')
    secCrawler.filing_10K(CompanyName, CIK, Date, '1')

Path of the directory: D:\seclogdown/
http://www.sec.gov/Archives/edgar/data/1000209/000119312517082178/0001193125-17-082178.txt
'SecCrawler' object has no attribute 'ua'
http://www.sec.gov/Archives/edgar/data/1000228/000100022817000011/0001000228-17-000011.txt
'SecCrawler' object has no attribute 'ua'
http://www.sec.gov/Archives/edgar/data/1000229/000100022917000025/0001000229-17-000025.txt
'SecCrawler' object has no attribute 'ua'
